In [153]:
import pandas as pd
import numpy as np
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [154]:
# Reading all the csv files into dfs
address = pd.read_csv('address_details.csv', dtype={'OFFENDER NC DOC ID NUMBER': str})
county = pd.read_csv('county_details.csv', dtype={'OFFENDER NC DOC ID NUMBER': str})
info = pd.read_csv('incar_info.csv', dtype={'INMATE DOC NUMBER': str})
sen = pd.read_csv('sentence_and_release_details.csv', dtype={'INMATE DOC NUMBER': str})

# Just keeping those rows which have common DOC ID in all dfs
id1 = set(address["OFFENDER NC DOC ID NUMBER"])
id2 = set(county["OFFENDER NC DOC ID NUMBER"])
id3 = set(info["INMATE DOC NUMBER"])
id4 = set(sen["INMATE DOC NUMBER"])

common_ids = id1.intersection(id2, id3, id4)

common_ids_list = list(common_ids)

common_ids_series = pd.Series(common_ids_list)
address = address[address['OFFENDER NC DOC ID NUMBER'].isin(common_ids_list)]
county = county[county['OFFENDER NC DOC ID NUMBER'].isin(common_ids_list)]
info = info[info['INMATE DOC NUMBER'].isin(common_ids_list)]
sen = sen[sen['INMATE DOC NUMBER'].isin(common_ids_list)]

# changing the name of DOC ID columns
county.rename(columns={'OFFENDER NC DOC ID NUMBER': 'INMATE DOC NUMBER'}, inplace=True)
address.rename(columns={'OFFENDER NC DOC ID NUMBER': 'INMATE DOC NUMBER'}, inplace=True)

In [155]:
county_data = {
    "001": "Alamance",
    "002": "Alexander",
    "003": "Alleghany",
    "004": "Anson",
    "005": "Ashe",
    "006": "Avery",
    "007": "Beaufort",
    "008": "Bertie",
    "009": "Bladen",
    "010": "Brunswick",
    "011": "Buncombe",
    "012": "Burke",
    "013": "Cabarrus",
    "014": "Caldwell",
    "015": "Camden",
    "016": "Carteret",
    "017": "Caswell",
    "018": "Catawba",
    "019": "Chatham",
    "020": "Cherokee",
    "021": "Chowan",
    "022": "Clay",
    "023": "Cleveland",
    "024": "Columbus",
    "025": "Craven",
    "026": "Cumberland",
    "027": "Currituck",
    "028": "Dare",
    "029": "Davidson",
    "030": "Davie",
    "031": "Duplin",
    "032": "Durham",
    "033": "Edgecombe",
    "034": "Forsyth",
    "035": "Franklin",
    "036": "Gaston",
    "037": "Gates",
    "038": "Graham",
    "039": "Granville",
    "040": "Greene",
    "041": "Guilford",
    "042": "Halifax",
    "043": "Harnett",
    "044": "Haywood",
    "045": "Henderson",
    "046": "Hertford",
    "047": "Hoke",
    "048": "Hyde",
    "049": "Iredell",
    "050": "Jackson",
    "051": "Johnston",
    "052": "Jones",
    "053": "Lee",
    "054": "Lenoir",
    "055": "Lincoln",
    "056": "Macon",
    "057": "Madison",
    "058": "Martin",
    "059": "McDowell",
    "060": "Mecklenburg",
    "061": "Mitchell",
    "062": "Montgomery",
    "063": "Moore",
    "064": "Nash",
    "065": "New Hanover",
    "066": "Northampton",
    "067": "Onslow",
    "068": "Orange",
    "069": "Pamlico",
    "070": "Pasquotank",
    "071": "Pender",
    "072": "Perquimans",
    "073": "Person",
    "074": "Pitt",
    "075": "Polk",
    "076": "Randolph",
    "077": "Richmond",
    "078": "Robeson",
    "079": "Rockingham",
    "080": "Rowan",
    "081": "Rutherford",
    "082": "Sampson",
    "083": "Scotland",
    "084": "Stanly",
    "085": "Stokes",
    "086": "Surry",
    "087": "Swain",
    "088": "Transylvania",
    "089": "Tyrrell",
    "090": "Union",
    "091": "Vance",
    "092": "Wake",
    "093": "Warren",
    "094": "Washington",
    "095": "Watauga",
    "096": "Wayne",
    "097": "Wilkes",
    "098": "Wilson",
    "099": "Yadkin",
    "100": "Yancey",
    "Out-of-state": "999"
}

In [156]:
!pip install census us

In [157]:
from census import Census
from us import states

API_KEY = '8c7d0af17c7bac1402d00779c41820156439acdb'
c = Census(API_KEY)

# Get population data for North Carolina from the latest available year (let's say 2019 as an example)
nc_data = c.acs5.state(('NAME', 'B01003_001E'), states.NC.fips, year=2019)

# Extract and print the data
for data in nc_data:
    print(f"State: {data['NAME']}")
    print(f"Population: {data['B01003_001E']}")

State: North Carolina
Population: 10264876.0


In [158]:
def fetch_data_by_county_acs(Year):
    # Define fields to fetch
    # Note: ACS might use different field codes than the decennial census!
    fields = (
        'NAME', 'B01001_001E', 'B02001_002E', 'B02001_003E', 'B02001_004E', 
        'B02001_005E', 'B01001_002E', 'B01001_026E', 'B03002_003E',  'B19013_001E', 'B19013A_001E', 
        'B19013B_001E', 'B19013C_001E', 'B19013D_001E', 'B19013E_001E', 'B19013F_001E', 'B19013G_001E', 
        'B19013I_001E', 'B16001_002E', 'B16001_003E', 'B16001_006E', 'B16001_009E', 'B16001_012E'
   
    )

    # Fetch data from ACS 2005-2009 5-year estimates
    data = c.acs5.state_county(fields, states.NC.fips, '*', year=Year)  # 5-Year Estimates for 2005-2009 are labeled as 2009

    # Create a list to store the structured data
    rows = []
    for entry in data:
        row = {
            'County': entry['NAME'],
            'Total Population': entry['B01001_001E'],
            'White Population': entry['B02001_002E'],
            'Black or African American Population': entry['B02001_003E'],
            'American Indian and Alaska Native Population': entry['B02001_004E'],
            'Asian Population': entry['B02001_005E'],
            'Male Population': entry['B01001_002E'],
            'Female Population': entry['B01001_026E'],
            'Hispanic or Latino Population': entry['B03002_003E'],
            'Median Household Income (Overall)': entry['B19013_001E'],
            'Median Household Income (White Alone)': entry['B19013A_001E'],
            'Median Household Income (Black or African American Alone)': entry['B19013B_001E'],
            'Median Household Income (American Indian and Alaska Native Alone)': entry['B19013C_001E'],
            'Median Household Income (Asian Alone)': entry['B19013D_001E'],
            'Median Household Income (Native Hawaiian and Other Pacific Islander Alone)': entry['B19013E_001E'],
            'Median Household Income (Some Other Race Alone)': entry['B19013F_001E'],
            'Median Household Income (Two or More Races)': entry['B19013G_001E'],
            'Median Household Income (Hispanic or Latino)': entry['B19013I_001E'],
            'Speak Only English at Home (Population 5 years and over)': entry['B16001_002E'],
            'Speak Spanish at Home (Population 5 years and over)': entry['B16001_003E'],
            'Speak Other Indo-European Languages at Home (Population 5 years and over)': entry['B16001_006E'],
            'Speak Asian and Pacific Island Languages at Home (Population 5 years and over)': entry['B16001_009E'],
            'Speak Other Languages at Home (Population 5 years and over)': entry['B16001_012E']
        }
        rows.append(row)

    # Convert the list to a pandas DataFrame
    df = pd.DataFrame(rows)
    df['Year'] = Year
    return df

In [159]:
# Define the column names
columns = ['County', 'Total Population', 'White Population',
       'Black or African American Population',
       'American Indian and Alaska Native Population', 'Asian Population',
       'Male Population', 'Female Population', 'Hispanic or Latino Population',
       'Median Household Income (Overall)',
       'Median Household Income (White Alone)',
       'Median Household Income (Black or African American Alone)',
       'Median Household Income (American Indian and Alaska Native Alone)',
       'Median Household Income (Asian Alone)',
       'Median Household Income (Native Hawaiian and Other Pacific Islander Alone)',
       'Median Household Income (Some Other Race Alone)',
       'Median Household Income (Two or More Races)',
       'Median Household Income (Hispanic or Latino)',
       'Speak Only English at Home (Population 5 years and over)',
       'Speak Spanish at Home (Population 5 years and over)',
       'Speak Other Indo-European Languages at Home (Population 5 years and over)',
       'Speak Asian and Pacific Island Languages at Home (Population 5 years and over)',
       'Speak Other Languages at Home (Population 5 years and over)', 'Year']

# Create an empty DataFrame with the specified columns
nc = pd.DataFrame(columns=columns)

for year in range(2009, 2022):
    nc_temp = fetch_data_by_county_acs(year)
    global nc
    nc = pd.concat([nc, nc_temp], ignore_index=True)

In [160]:
nc.to_csv('NC_Census_By_County_Population_and_Income.csv', index=False)

In [121]:
# county.columns

In [122]:
# info.columns

In [161]:
# Merge the DataFrames on 'ID NUMBER'
merged_df = county.merge(info[['INMATE DOC NUMBER', 'INMATE LAST NAME', 'INMATE FIRST NAME', 'INMATE MIDDLE INITIAL', 'INMATE NAME SUFFIX', 'INMATE NAME SOUNDEX CODE', 'INMATE GENDER CODE', 'INMATE RACE CODE', 'INMATE BIRTH DATE', 'INMATE ETHNIC AFFILIATION', 'INMATE ADMISSION DATE']], on='INMATE DOC NUMBER', how='left')

In [162]:
# Assuming you have merged_df from the previous step
def extract_year(date_str):
    try:
        return pd.to_datetime(date_str).year
    except pd.errors.OutOfBoundsDatetime:
        return np.nan

merged_df['Year of Sentence'] = merged_df['SENTENCE EFFECTIVE(BEGIN) DATE'].apply(extract_year)

In [125]:
# merged_df.head()

In [163]:
filtered_df = merged_df[(merged_df['Year of Sentence'] >= 2009) & (merged_df['Year of Sentence'] <= 2021)]
filtered_df = filtered_df[filtered_df['COUNTY OF CONVICTION CODE'] != '']
filtered_df = filtered_df[filtered_df['COUNTY OF CONVICTION CODE'] != 'OUT OF STATE']
filtered_df['INMATE RACE CODE'] = filtered_df['INMATE RACE CODE'].replace('UNKNOWN', 'OTHER')


In [164]:
filtered_df['County'] = filtered_df['COUNTY OF CONVICTION CODE'].apply(lambda x: f'{x.strip().lower().title()} County, North Carolina')

In [128]:
# nc['County'].unique()

In [129]:
# filtered_df['County'].unique()

In [165]:
filtered_df['INMATE RACE CODE'].unique()

array(['WHITE', 'BLACK/AFRICAN AMERICAN',
       'AMERICAN INDIAN/ALASKAN NATIVE', 'OTHER', 'ASIAN/ASIAN AMERICAN',
       'HAWAIIAN/PACIFIC ISLANDER'], dtype=object)

In [166]:
filtered_df.drop('Unnamed: 0', axis=1, inplace=True)

In [167]:
filtered_df.count()

INMATE DOC NUMBER                 929921
COUNTY OF CONVICTION CODE         929921
PRIMARY OFFENSE CODE              929921
DATE OFFENSE COMMITTED - BEGIN    929921
DATE OFFENSE COMMITTED - END      929921
PRIMARY FELONY/MISDEMEANOR CD.    929921
PRIOR RCD. POINTS/CONVICTIONS     929921
MINIMUM SENTENCE LENGTH           929921
MAXIMUM SENTENCE LENGTH           929921
CREDITS FOR JAIL DAYS SERVED      929921
SENTENCE CONVICTION DATE          929921
SENTENCE EFFECTIVE(BEGIN) DATE    929921
DATE OF LAST UPDATE               929921
INMATE LAST NAME                  929906
INMATE FIRST NAME                 929921
INMATE MIDDLE INITIAL             751799
INMATE NAME SUFFIX                 55280
INMATE NAME SOUNDEX CODE          496610
INMATE GENDER CODE                929921
INMATE RACE CODE                  929921
INMATE BIRTH DATE                 929921
INMATE ETHNIC AFFILIATION         929135
INMATE ADMISSION DATE             929921
Year of Sentence                  929921
County          

In [168]:
# Group by 'Year of Sentence' and 'Formatted County', and sum the races
grouped = filtered_df.groupby(['Year of Sentence', 'County'])['INMATE RACE CODE'].value_counts().unstack(fill_value=0)

# Reset the index to make 'Year of Sentence' and 'Formatted County' regular columns
grouped = grouped.reset_index()
grouped['Year of Sentence'] = grouped['Year of Sentence'].astype(int)

In [134]:
# grouped.head(100)

In [169]:
grouped.to_csv('NC_By_Year_And_County_Offences_Counts.csv', index=False)